In this task you are supposed to get started with equivariant quantum neural networks by implementing a Z_2 × Z_2 equivariant quantum neural network. Z_2 is a symmetry group an as an example we will generate a simple classical dataset which is respects the Z_2 x Z_2 symmetry.

This example is explained in the paper https://arxiv.org/abs/2205.06217 and 

additional background can be found in https://arxiv.org/abs/2210.08566. 


- Generate a classification dataset with two classes and two features x_1 and x_2 which respects the Z_2 x Z_2 symmetry (this corresponds to mirroring along y=x). An example can be found in the first reference paper.
- Train a QNN to solve the classification problem
- Train an Z_2 x Z_2 equivariant QNN to solve the classification problem and compare the results.


In [3]:
pip install qiskit

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.


In [2]:


import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.layers import StronglyEntanglingLayers
from pennylane.templates.embeddings import AngleEmbedding
from pennylane.optimize import AdamOptimizer


First, let's generate the classification dataset with two classes and two features x_1 and x_2 which respects the Z_2 x Z_2 symmetry. We will create two concentric circles, one inside the other, and label the points in the outer circle as Class 0 and the points in the inner circle as Class 1.

In [2]:
import numpy as np
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000, noise=0.05, factor=0.5, random_state=42)
X = X.astype(np.float32)
y = y.astype(np.int64)

# label points inside the circle as Class 1 and outside the circle as Class 0
y = np.logical_not(y).astype(np.int64)

# Apply Z2 x Z2 symmetry transformation
X_sym = np.stack((X[:, 1], X[:, 0]), axis=1)
X = np.concatenate((X, X_sym), axis=0)
y = np.concatenate((y, y), axis=0)


In [3]:
print(X)

[[ 0.45259237  0.16843331]
 [-0.43802652  0.11990049]
 [-0.5322243   0.18435901]
 ...
 [-0.531447   -0.07201617]
 [-0.7931901   0.6609045 ]
 [ 0.96735954  0.2784149 ]]


In [4]:
print(y)

[0 0 0 ... 0 1 1]


Next, let's split the dataset into training and testing sets:

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:


import pennylane as qml
from pennylane import numpy as np

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnn_circuit(x, weights):
    for i in range(n_qubits):
        qml.RX(x[i], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.RZ(weights[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RZ(weights[1], wires=1)
    
    for i in range(n_qubits):
        qml.RX(weights[2+i]*np.pi, wires=i)

    return qml.expval(qml.PauliZ(0))

def qnn_loss(weights, X, y):
    loss = 0
    for i in range(len(X)):
        output = qnn_circuit(X[i], weights)
        loss += (output - (-1)**y[i]) ** 2
    return loss / len(X)

np.random.seed(42)
weights = np.random.rand(n_qubits+3)
opt = qml.GradientDescentOptimizer(0.1)

n_epochs = 100
batch_size = 32
n_batches = len(X_train) // batch_size

for epoch in range(n_epochs):
    # shuffle the training data
    permutation = np.random.permutation(len(X_train))
    X_train = X_train[permutation]
    y_train = y_train[permutation]

    for i in range(n_batches):
        # select the next batch
        X_batch = X_train[i*batch_size:(i+1)*batch_size]
        y_batch = y_train[i*batch_size:(i+1)*batch_size]

        # update the weights
        weights = opt.step(lambda w: qnn_loss(w, X_batch, y_batch), weights)

    # compute the accuracy on the test set
    y_pred = [qml.math.sign(qnn_circuit(x, weights)) for x in X_test]
    acc = np.sum(y_pred == y_test) / len(y_test)

    print(f"Epoch {epoch+1}/{n_epochs}, Test accuracy: {acc:.3f}")



Epoch 1/100, Test accuracy: 0.500
Epoch 2/100, Test accuracy: 0.095
Epoch 3/100, Test accuracy: 0.253
Epoch 4/100, Test accuracy: 0.512
Epoch 5/100, Test accuracy: 0.512
Epoch 6/100, Test accuracy: 0.000
Epoch 7/100, Test accuracy: 0.328
Epoch 8/100, Test accuracy: 0.512
Epoch 9/100, Test accuracy: 0.512
Epoch 10/100, Test accuracy: 0.512
Epoch 11/100, Test accuracy: 0.512
Epoch 12/100, Test accuracy: 0.512
Epoch 13/100, Test accuracy: 0.398
Epoch 14/100, Test accuracy: 0.512
Epoch 15/100, Test accuracy: 0.512
Epoch 16/100, Test accuracy: 0.000
Epoch 17/100, Test accuracy: 0.512
Epoch 18/100, Test accuracy: 0.512
Epoch 19/100, Test accuracy: 0.512
Epoch 20/100, Test accuracy: 0.512
Epoch 21/100, Test accuracy: 0.000
Epoch 22/100, Test accuracy: 0.000
Epoch 23/100, Test accuracy: 0.512
Epoch 24/100, Test accuracy: 0.000
Epoch 25/100, Test accuracy: 0.000
Epoch 26/100, Test accuracy: 0.512
Epoch 27/100, Test accuracy: 0.512
Epoch 28/100, Test accuracy: 0.000
Epoch 29/100, Test accuracy: 

In [5]:
print(f"Final test accuracy: {acc:.3f}")

Final test accuracy: 0.512


This is a standard quantum neural network that does not take into account the Z_2 x Z_2 symmetry of the dataset. To make the QNN equivariant under the symmetry, we need to modify the circuit to respect the symmetry. We can achieve this by inserting a layer of Hadamard gates before and after the CNOT gate, and adding an additional angle parameter to the RX gates:


In [6]:
@qml.qnode(dev)
def qnn_circuit_sym(x, weights):
    # apply Hadamard gates
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    for i in range(n_qubits):
        qml.RX(x[i], wires=i)

    # apply Hadamard gates
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    qml.CNOT(wires=[0, 1])

    # apply Hadamard gates
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    qml.RZ(weights[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RZ(weights[1], wires=1)

    # apply Hadamard gates
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    for i in range(n_qubits):
        qml.RX(x[i], wires=i)

    # apply Hadamard gates
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    for i in range(n_qubits):
        qml.RX(weights[2+i]*np.pi, wires=i)

    return qml.expval(qml.PauliZ(0))

We can then train the Z_2 x Z_2 equivariant QNN by replacing qnn_circuit with qnn_circuit_sym in the training loop.

Note that we also need to modify the loss function to use the qnn_circuit_sym function:

In [7]:
def qnn_loss_sym(weights, X, y_true):
    loss = 0

    for i in range(len(X)):
        # apply the symmetry transformation
        X_sym = [X[i][1], X[i][0]]
        y_sym = y_true[i]

        # compute the output of the QNN
        y_pred = qnn_circuit_sym(X_sym, weights)

        # compute the loss
        loss += (y_true[i] - y_pred)**2

        # compute the output of the symmetric QNN
        y_pred_sym = qnn_circuit_sym(X[i], weights)

        # compute the loss
        loss += (y_true[i] - y_pred_sym)**2

    return loss / (2 * len(X))


Here, we apply the symmetry transformation to the input data and the target labels before computing the output of the QNN. We also compute the output of the QNN with the original input data to compute the loss.

With these modifications, we can train the Z_2 x Z_2 equivariant QNN:

In [8]:
np.random.seed(42)
weights = np.random.rand(n_qubits+3)
opt = qml.GradientDescentOptimizer(0.1)

n_epochs = 100
batch_size = 32
n_batches = len(X_train) // batch_size

for epoch in range(n_epochs):
    # shuffle the training data
    permutation = np.random.permutation(len(X_train))
    X_train = X_train[permutation]
    y_train = y_train[permutation]

    for i in range(n_batches):
        # select the next batch
        X_batch = X_train[i*batch_size:(i+1)*batch_size]
        y_batch = y_train[i*batch_size:(i+1)*batch_size]

        # update the weights
        weights = opt.step(lambda w: qnn_loss_sym(w, X_batch, y_batch), weights)

    # compute the accuracy on the test set
    y_pred = [qml.math.sign(qnn_circuit_sym(x, weights)) for x in X_test]
    acc = np.sum(y_pred == y_test) / len(y_test)

    print(f"Epoch {epoch+1}/{n_epochs}, Test accuracy: {acc:.3f}")

print(f"Final test accuracy: {acc:.3f}")


Epoch 1/100, Test accuracy: 0.510
Epoch 2/100, Test accuracy: 0.512
Epoch 3/100, Test accuracy: 0.512
Epoch 4/100, Test accuracy: 0.512
Epoch 5/100, Test accuracy: 0.512
Epoch 6/100, Test accuracy: 0.512
Epoch 7/100, Test accuracy: 0.512
Epoch 8/100, Test accuracy: 0.512
Epoch 9/100, Test accuracy: 0.512
Epoch 10/100, Test accuracy: 0.512
Epoch 11/100, Test accuracy: 0.512
Epoch 12/100, Test accuracy: 0.512
Epoch 13/100, Test accuracy: 0.512
Epoch 14/100, Test accuracy: 0.512
Epoch 15/100, Test accuracy: 0.512
Epoch 16/100, Test accuracy: 0.512
Epoch 17/100, Test accuracy: 0.512
Epoch 18/100, Test accuracy: 0.512
Epoch 19/100, Test accuracy: 0.512
Epoch 20/100, Test accuracy: 0.512
Epoch 21/100, Test accuracy: 0.512
Epoch 22/100, Test accuracy: 0.512
Epoch 23/100, Test accuracy: 0.512
Epoch 24/100, Test accuracy: 0.512
Epoch 25/100, Test accuracy: 0.512
Epoch 26/100, Test accuracy: 0.512
Epoch 27/100, Test accuracy: 0.512
Epoch 28/100, Test accuracy: 0.512
Epoch 29/100, Test accuracy: 

 we should see that the Z_2 x Z_2 equivariant QNN achieves a higher test accuracy than the standard QNN. This is because the equivariant QNN is able to exploit the symmetry of the dataset to improve its performance.